In [1]:
# 导入OpenAI库，并创建OpenAI客户端
from dotenv import load_dotenv
load_dotenv()
from openai import OpenAI
client = OpenAI()

In [2]:
# 导入数据文件
import pandas as pd
file_path = 'sales_data.csv'
sales_data = pd.read_csv(file_path)
sales_data.head()

,日期,零基础学机器学习,数据分析咖哥十话,GPT图解
0,31/3/2022,303.032760,985.615033,909.762701
1,30/6/2022,504.892977,871.129109,1023.037873
2,30/9/2022,576.723513,1035.950790,1080.552675
3,31/12/2022,670.501822,1068.182030,1148.976637
4,31/3/2023,766.792751,718.352429,1204.730960


In [3]:
# 创建文件
file = client.files.create(
  file=open(file_path, "rb"),
  purpose='assistants',
)

# 创建一个包含这个文件的助手
assistant = client.beta.assistants.create(
  instructions="作为一名数据科学助理，当给定数据和一个查询时，你能编写适当的代码并创建适当的可视化。",
  model="gpt-4-0125-preview",
  tools=[
    {"type": "code_interpreter"}
  ],
  tool_resources={
    "code_interpreter": {
      "file_ids": [file.id]  # Here we add the file id
    }
  }
)
print(assistant)

Assistant(id='asst_Brzqgi6j8cjuvQtRqMlCZxAO', created_at=1714004967, description=None, instructions='作为一名数据科学助理，当给定数据和一个查询时，你能编写适当的代码并创建适当的可视化。', metadata={}, model='gpt-4-0125-preview', name=None, object='assistant', tools=[CodeInterpreterTool(type='code_interpreter')], tool_resources=ToolResources(code_interpreter=ToolResourcesCodeInterpreter(file_ids=['file-vREOueLEpmTVGmNjF8KMaYrq']), file_search=None), top_p=1.0, temperature=1.0, response_format='auto')


In [4]:
# 创建对话线程并运行
thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": "计算从2022年到2025年每个季度的总销售额，并通过不同的产品将其可视化为折线图，产品线条颜色分别为红，蓝，绿。",
      "attachments": [
        {
          "file_id": file.id,
          "tools": [
            { "type": "code_interpreter" }
          ]
        }
      ]
    }
  ]
)
print(thread)

Thread(id='thread_XU3pjbxtGi4h2KyNXzg5nOA4', created_at=1714004967, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=ToolResourcesCodeInterpreter(file_ids=['file-vREOueLEpmTVGmNjF8KMaYrq']), file_search=None))


In [5]:
# 创建Run来运行和助手的对话
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)
print(run)

Run(id='run_Ss6o8VT9qDYsq1vtf9CSK38L', assistant_id='asst_Brzqgi6j8cjuvQtRqMlCZxAO', cancelled_at=None, completed_at=None, created_at=1714004968, expires_at=1714005568, failed_at=None, incomplete_details=None, instructions='作为一名数据科学助理，当给定数据和一个查询时，你能编写适当的代码并创建适当的可视化。', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4-0125-preview', object='thread.run', required_action=None, response_format='auto', started_at=None, status='queued', thread_id='thread_XU3pjbxtGi4h2KyNXzg5nOA4', tool_choice='auto', tools=[CodeInterpreterTool(type='code_interpreter')], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), usage=None, temperature=1.0, top_p=1.0, tool_resources={})


In [6]:
# 检查并等待可视化完成
import time
while True:
    messages = client.beta.threads.messages.list(thread_id=thread.id)
    try:
        # 检查是否创建了图像
        messages.data[0].content[0].image_file
        # 等待运行完成
        time.sleep(5)
        print('图表已创建！')
        if messages.data and messages.data[0].content:
            print('当前Message:', messages.data[0].content[0])
        break
    except:
        time.sleep(10)
        print('您的Assistant正在努力做图表呢...')
        if messages.data and messages.data[0].content:
            print('当前Message:', messages.data[0].content[0])

您的Assistant正在努力做图表呢...
当前Message: TextContentBlock(text=Text(annotations=[], value='计算从2022年到2025年每个季度的总销售额，并通过不同的产品将其可视化为折线图，产品线条颜色分别为红，蓝，绿。'), type='text')
您的Assistant正在努力做图表呢...
您的Assistant正在努力做图表呢...
当前Message: TextContentBlock(text=Text(annotations=[], value='数据确实包含从2022年3月31日到2025年12月31日的日期范围，符合要求。接下来，我们将计算每个季度的总销售额，并通过折线图进行可视化，使用指定的颜色表示不同的产品。'), type='text')
您的Assistant正在努力做图表呢...
当前Message: TextContentBlock(text=Text(annotations=[], value='数据确实包含从2022年3月31日到2025年12月31日的日期范围，符合要求。接下来，我们将计算每个季度的总销售额，并通过折线图进行可视化，使用指定的颜色表示不同的产品。'), type='text')
您的Assistant正在努力做图表呢...
图表已创建！
当前Message: ImageFileContentBlock(image_file=ImageFile(file_id='file-yMlgsnfVuwi2k5ybnvHfjpcV'), type='image_file')


In [7]:
# 转换输出文件为PNG格式
def convert_file_to_png(file_id, write_path):
    data = client.files.content(file_id)
    data_bytes = data.read()
    with open(write_path, "wb") as file:
        file.write(data_bytes)

plot_file_id = messages.data[0].content[0].image_file.file_id
image_path = "咖哥图书销量.png"
convert_file_to_png(plot_file_id,image_path)

# 上传图表
plot_file = client.files.create(
  file=open(image_path, "rb"),
  purpose='assistants'
)

In [8]:
# 看看助手整个的思维和行动过程
messages = client.beta.threads.messages.list(thread_id=thread.id)
[message.content[0] for message in messages.data]

[ImageFileContentBlock(image_file=ImageFile(file_id='file-yMlgsnfVuwi2k5ybnvHfjpcV'), type='image_file'),
 TextContentBlock(text=Text(annotations=[], value='数据确实包含从2022年3月31日到2025年12月31日的日期范围，符合要求。接下来，我们将计算每个季度的总销售额，并通过折线图进行可视化，使用指定的颜色表示不同的产品。'), type='text'),
 TextContentBlock(text=Text(annotations=[], value='文件包含了从2022年到2025年每个季度的销售数据，列名为“日期”，“零基础学机器学习”，“数据分析咖哥十话”，以及“GPT图解”。接下来，我们将：\n\n1. 检查日期范围以确保数据覆盖从2022年到2025年。\n2. 计算每个季度的总销售额。\n3. 通过不同产品将总销售额可视化为折线图，产品线条颜色分别为红，蓝，绿。\n\n让我们从检查日期范围开始。'), type='text'),
 TextContentBlock(text=Text(annotations=[], value='首先，我将检查上传的文件内容，以了解数据的格式和结构。安装好文件之后，我们可以继续计算总销售额，并按要求进行可视化。'), type='text'),
 TextContentBlock(text=Text(annotations=[], value='计算从2022年到2025年每个季度的总销售额，并通过不同的产品将其可视化为折线图，产品线条颜色分别为红，蓝，绿。'), type='text')]

In [9]:
# 定义提交用户的消息的函数
def submit_message_wait_completion(assistant_id, thread, user_message, file_ids=None):
    # 检查并等待活跃的Run完成
    for run in client.beta.threads.runs.list(thread_id=thread.id).data:
        if run.status == 'in_progress':
            print(f"等待Run {run.id} 完成...")
            while True:
                run_status = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id).status
                if run_status in ['succeeded', 'failed']:
                    break
                time.sleep(5)  # 等待5秒后再次检查状态

    # 提交消息
    params = {
        'thread_id': thread.id,
        'role': 'user',
        'content': user_message,
    }
    # 设置attachments
    if file_ids:        
        attachments = [{"file_id": file_id, "tools": [ {"type": "code_interpreter"}]} for file_id in file_ids]
        params['attachments'] = attachments
    client.beta.threads.messages.create(**params)

    # 创建Run
    run = client.beta.threads.runs.create(thread_id=thread.id, assistant_id=assistant_id)
    return run 


In [10]:
# 使用submit_message函数, 来发送请求，让助手生成洞察
submit_message_wait_completion(assistant.id, thread, "请根据你刚才创建的图表，给我两个约20字的句子，描述最重要的洞察。这将用于幻灯片展示，揭示出数据背后的'秘密'。")

Run(id='run_tYw85XwXuoQIbvq9zmSuWYmT', assistant_id='asst_Brzqgi6j8cjuvQtRqMlCZxAO', cancelled_at=None, completed_at=None, created_at=1714005029, expires_at=1714005629, failed_at=None, incomplete_details=None, instructions='作为一名数据科学助理，当给定数据和一个查询时，你能编写适当的代码并创建适当的可视化。', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4-0125-preview', object='thread.run', required_action=None, response_format='auto', started_at=None, status='queued', thread_id='thread_XU3pjbxtGi4h2KyNXzg5nOA4', tool_choice='auto', tools=[CodeInterpreterTool(type='code_interpreter')], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), usage=None, temperature=1.0, top_p=1.0, tool_resources={})

In [11]:
# 获取对话线程的响应
def get_response(thread):
    return client.beta.threads.messages.list(thread_id=thread.id)

# 等待响应并打印洞察
time.sleep(10) # 假设助理需要一些时间来生成洞察
response = get_response(thread)
bullet_points = response.data[0].content[0].text.value
print(bullet_points)

1. "GPT图解"的销售额稳步上升，显示出强劲的市场需求和增长潜力。
2. "数据分析咖哥十话"虽有起伏，但整体上呈现出稳定增长的趋势。


In [12]:
# 根据见解生成标题 
submit_message_wait_completion(assistant.id, thread, "根据你创建的情节和要点，为幻灯片想一个非常简短的标题。它应该只反映你得出的主要见解。")


Run(id='run_LBxNrn3TiRFJGsQJYeUSSwCy', assistant_id='asst_Brzqgi6j8cjuvQtRqMlCZxAO', cancelled_at=None, completed_at=None, created_at=1714005040, expires_at=1714005640, failed_at=None, incomplete_details=None, instructions='作为一名数据科学助理，当给定数据和一个查询时，你能编写适当的代码并创建适当的可视化。', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4-0125-preview', object='thread.run', required_action=None, response_format='auto', started_at=None, status='queued', thread_id='thread_XU3pjbxtGi4h2KyNXzg5nOA4', tool_choice='auto', tools=[CodeInterpreterTool(type='code_interpreter')], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), usage=None, temperature=1.0, top_p=1.0, tool_resources={})

In [13]:
# 等待响应并打印标题
time.sleep(10) # 等待助理生成标题
response = get_response(thread)
title = response.data[0].content[0].text.value
print(title)

"强劲增长：GPT图解与数据分析的市场突破"


In [14]:
# 提供花语秘境公司的说明
company_summary = "我们是网络鲜花批发商，但是我们董事长也写IT图书！"

# 使用DALL-E 3生成图片
response = client.images.generate(
  model='dall-e-3',
  prompt=f"根据这个公司概述 {company_summary}, \
           创建一张展示成长和前进道路的启发性照片。这将用于季度销售规划会议",
       size="1024x1024",
       quality="hd",
       n=1
)
image_url = response.data[0].url

# 获取DALL-E 3生成的图片
import requests
dalle_img_path = '花语秘境咖哥.png'
img = requests.get(image_url)

# 将图片保存到本地
with open(dalle_img_path,'wb') as file:
  file.write(img.content)

# 上传图片提供给助手做为PPT素材
dalle_file = client.files.create(
  file=open(dalle_img_path, "rb"),
  purpose='assistants'
)

In [15]:
title_template = """
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.enum.text import PP_PARAGRAPH_ALIGNMENT
from pptx.dml.color import RGBColor

# 创建新的演示文稿对象
prs = Presentation()

# 添加一个空白的幻灯片布局
blank_slide_layout = prs.slide_layouts[6]
slide = prs.slides.add_slide(blank_slide_layout)

# 将幻灯片的背景颜色设置为黑色
background = slide.background
fill = background.fill
fill.solid()
fill.fore_color.rgb = RGBColor(0, 0, 0)

# 在幻灯片左侧添加图片，上下留有边距
left = Inches(0)
top = Inches(0)
height = prs.slide_height
width = prs.slide_width * 3/5
pic = slide.shapes.add_picture(image_path, left, top, width=width, height=height)

# 在较高位置添加标题文本框
left = prs.slide_width * 3/5
top = Inches(2)
width = prs.slide_width * 2/5
height = Inches(1)
title_box = slide.shapes.add_textbox(left, top, width, height)
title_frame = title_box.text_frame
title_p = title_frame.add_paragraph()
title_p.text = title_text
title_p.font.bold = True
title_p.font.size = Pt(38)
title_p.font.color.rgb = RGBColor(255, 255, 255)
title_p.alignment = PP_PARAGRAPH_ALIGNMENT.CENTER

# 添加副标题文本框
left = prs.slide_width * 3/5
top = Inches(3)
width = prs.slide_width * 2/5
height = Inches(1)
subtitle_box = slide.shapes.add_textbox(left, top, width, height)
subtitle_frame = subtitle_box.text_frame
subtitle_p = subtitle_frame.add_paragraph()
subtitle_p.text = subtitle_text
subtitle_p.font.size = Pt(22)
subtitle_p.font.color.rgb = RGBColor(255, 255, 255)
subtitle_p.alignment = PP_PARAGRAPH_ALIGNMENT.CENTER
"""

data_vis_template = """
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.enum.text import PP_PARAGRAPH_ALIGNMENT
from pptx.dml.color import RGBColor

# 创建新的演示文稿对象
prs = Presentation()

# 添加一个空白的幻灯片布局
blank_slide_layout = prs.slide_layouts[6]
slide = prs.slides.add_slide(blank_slide_layout)

# 将幻灯片的背景颜色设置为黑色
background = slide.background
fill = background.fill
fill.solid()
fill.fore_color.rgb = RGBColor(0, 0, 0)

# 定义占位符
image_path = data_vis_img
title_text = "提升利润：在线销售与直销优化的主导地位"
bullet_points = "• 在线销售在各个季度中始终领先于盈利能力，表明了强大的数字市场存在。\n• 直销表现出波动，表明该渠道的表现变化和需要针对性改进的必要性。"

# 在幻灯片左侧添加图片占位符
left = Inches(0.2)
top = Inches(1.8)
height = prs.slide_height - Inches(3)
width = prs.slide_width * 3/5
pic = slide.shapes.add_picture(image_path, left, top, width=width, height=height)

# 添加覆盖整个宽度的标题文本
left = Inches(0)
top = Inches(0)
width = prs.slide_width
height = Inches(1)
title_box = slide.shapes.add_textbox(left, top, width, height)
title_frame = title_box.text_frame
title_frame.margin_top = Inches(0.1)
title_p = title_frame.add_paragraph()
title_p.text = title_text
title_p.font.bold = True
title_p.font.size = Pt(28)
title_p.font.color.rgb = RGBColor(255, 255, 255)
title_p.alignment = PP_PARAGRAPH_ALIGNMENT.CENTER

# 添加硬编码的“关键见解”文本和项目符号列表
left = prs.slide_width * 2/3
top = Inches(1.5)
width = prs.slide_width * 1/3
height = Inches(4.5)
insights_box = slide.shapes.add_textbox(left, top, width, height)
insights_frame = insights_box.text_frame
insights_p = insights_frame.add_paragraph()
insights_p.text = "关键见解："
insights_p.font.bold = True
insights_p.font.size = Pt(24)
insights_p.font.color.rgb = RGBColor(0, 128, 100)
insights_p.alignment = PP_PARAGRAPH_ALIGNMENT.LEFT
insights_frame.add_paragraph()


bullet_p = insights_frame.add_paragraph()
bullet_p.text = bullet_points
bullet_p.font.size = Pt(12)
bullet_p.font.color.rgb = RGBColor(255, 255, 255)
bullet_p.line_spacing = 1.5
"""

title_text = "花语秘境"
subtitle_text = "2025年销售大会"

submit_message_wait_completion(assistant.id,thread,f"使用包含的代码模板创建符合模板格式的PPTX幻灯片，但使用本消息中包含的图片、公司名称/标题和文件名/副标题：\
{title_template}。重要提示：在此第一张幻灯片中使用本消息中包含的图片文件作为image_path图像，并使用公司名称 {title_text} 作为title_text变量，\
  使用副标题文本 {subtitle_text} 作为subtitle_text变量。\
    接着，使用以下代码模板创建第二张幻灯片：{data_vis_template}，创建符合模板格式的PPTX幻灯片，但使用公司名称/标题和文件名/副标题：\
{data_vis_template}。重要提示：使用您之前在本线程中创建的第二个附图（折线图）作为data_vis_img图像，并使用您之前创建的数据可视化标题作为title_text变量，\
  使用您之前创建的见解项目符号列表作为bullet_points变量。将这两张幻灯片输出为.pptx文件。确保输出为两张幻灯片，每张幻灯片都符合本消息中给出的相应模板。",
              file_ids=[dalle_file.id, plot_file.id]
)


# 等待助手完成PPT创建任务
while True:
    try:
        response = get_response(thread)
        pptx_id = response.data[0].content[0].text.annotations[0].file_path.file_id
        print("成功检索到 pptx_id:", pptx_id)
        break
    except Exception as e:
        print("您的Assistant正在努力制作幻灯片...")
        time.sleep(10)

import io
pptx_id = response.data[0].content[0].text.annotations[0].file_path.file_id
ppt_file= client.files.content(pptx_id)
file_obj = io.BytesIO(ppt_file.read())
with open("咖哥花语秘境.pptx", "wb") as f:
    f.write(file_obj.getbuffer())


您的Assistant正在努力制作幻灯片...
您的Assistant正在努力制作幻灯片...
您的Assistant正在努力制作幻灯片...
您的Assistant正在努力制作幻灯片...
您的Assistant正在努力制作幻灯片...
您的Assistant正在努力制作幻灯片...
您的Assistant正在努力制作幻灯片...
您的Assistant正在努力制作幻灯片...
成功检索到 pptx_id: file-sMIsjd50FCApZCS9dZbhRbsb
